## 2. Data Preparation

Data: 29 de Maio de 2023

Esse notebook contém:

- Inicialização de modelos de vetorização de texto pré-treinados
- Conversão de dados em texto para vetor e salvamento.

### Notas:



#### a. Data Colection

Fazer:
- adaptar o código para coletar knowledge_area em inglês. Talvez tudo em inglês em um primeiro momento porque eu suponho que os modelos funcionem melhor.
- Eu também preciso da url para o artigo, salvar isso.
- Muitos valores nulos durante o parsing, por:
    ```array(['tde-28102015-135333', 'tde-02012016-095549',
       'tde-24112014-141242', 'tde-24052006-144856',
       'tde-02092021-143225', 'tde-16102009-154941',
       'tde-01122016-092008', 'tde-25092008-165152',
       'tde-16032021-150930', 'tde-09062003-162453',
       'tde-23082017-210426', 'tde-27052004-160210',
       'tde-10102022-092326', 'tde-24102016-141623',
       'tde-28062021-164000', 'tde-24052006-155058',
       'tde-07112018-100009', 'tde-18042018-084856',
       'tde-02062006-105722', 'tde-05062020-090351',
       'tde-20102010-151349', 'tde-09112022-122413',
       'tde-21022020-154150', 'tde-07022022-171023',
       'tde-01062020-085825', 'tde-01062020-090238',
       'tde-08082022-111403', 'tde-05112021-110843',
       'tde-10042023-155349', 'tde-10042023-155123',
       'tde-07092020-101055'], dtype=object)```

#### b. Embedding Models


- Uma explicacao sobre o sentence transformer pode ser encontrada abaixo. Notar que eu usei um modelo especifico chamado distilbert-base-nli-stsb-mean-tokens para gerar os embeddings. Talvez eu possa testar outros modelos do hugging face. 

##### Fazer: 
- Verificar com a Nina como eu posso avaliar os modelos.

##### Sidenote: Explicação de Sentence Transformers

sentence-transformers is a library that provides easy methods to compute embeddings (dense vector representations) for sentences, paragraphs and images. Texts are embedded in a vector space such that similar text is close, which enables applications such as semantic search, clustering, and retrieval.

Há vários modelos para serem testados e cada um é treinado de uma forma diferente ou em um conjunto de dados diferente o que dá características únicas ao modelo e aos embeddings gerados, permitindo que cada modelo possa ser usado para um propósito específico.

All models on the Hub come up with features:

An automatically generated model card with a description, example code snippets, architecture overview, and more.
Metadata tags that help for discoverability and contain information such as license.
An interactive widget you can use to play out with the model directly in the browser.
An Inference API that allows to make inference requests.

https://huggingface.co/docs/hub/sentence-transformers
> **Pergunta**: o que faz um modelo ser treinado para similaridade semântica textual? Qual a diferença no treinamento? Apenas o conjunto de dados?

#### c. Vector Search
- Como funciona cosine_similarity?

In [1]:
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# pre-trained sentence encoder
# distilbert (distilated version of bert model)
# stsb denotes that the model was trained for semantic textual similarity

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device='cuda')

2023-05-30 12:21:01.251072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 12:21:02.460173: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-30 12:21:02.460246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-30 12:21:02.460251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

### Data Selection

In [225]:
df = pd.read_json("../data/theses.json", lines=False)
df = df.transpose()
df.reset_index(inplace=True)

#### Null values

In [226]:
df[df.isnull().any(axis=1)]["index"].values

array(['tde-28102015-135333', 'tde-02012016-095549',
       'tde-24112014-141242', 'tde-24052006-144856',
       'tde-02092021-143225', 'tde-16102009-154941',
       'tde-01122016-092008', 'tde-25092008-165152',
       'tde-16032021-150930', 'tde-09062003-162453',
       'tde-23082017-210426', 'tde-27052004-160210',
       'tde-10102022-092326', 'tde-24102016-141623',
       'tde-28062021-164000', 'tde-24052006-155058',
       'tde-07112018-100009', 'tde-18042018-084856',
       'tde-02062006-105722', 'tde-05062020-090351',
       'tde-20102010-151349', 'tde-09112022-122413',
       'tde-21022020-154150', 'tde-07022022-171023',
       'tde-01062020-085825', 'tde-01062020-090238',
       'tde-08082022-111403', 'tde-05112021-110843',
       'tde-10042023-155349', 'tde-10042023-155123',
       'tde-07092020-101055'], dtype=object)

In [227]:
df.dropna(inplace=True)

#### Feature cleaning

Algumas instâncias dos dados estavam com knowledge_area = "Knowledge Area", exemplo:
https://www.teses.usp.br/teses/disponiveis/98/98132/tde-27072020-145806/en.php

In [228]:
df.loc[:, "knowledge_area"][df["knowledge_area"] == 'Área do Conhecimento'] = "Medicine"

#### Selection

In [229]:
data = df.loc[:,["title_en", "abstract_en", "keywords_en"]]
relevant_metadata = df.loc[:, ["index", "type", "author", "institute"]]

### Encoding

O código a seguir cria batches de descrições a partir do título, abstract e palavras-chave e solicita ao modelo que codifique as embeddings, que são apensadas a vectors ao final.

##### Exemplo de execução

In [230]:
row = data.iloc[0,:]
description = row.title_en + " " + row.abstract_en + " " + row.keywords_en
description

'Gel Dosimetry in Tridimensional Quality Control Radiotherapy Treatments with Intensity Modulated and SMART Technique (Simultaneous Modulated Accelerated Radiation Therapy). Intensity Modulated Radiation Therapy (IMRT) treatments in the radiotherapy practice made possible the simultaneously treatment of multiple targets at an accelerated treatment regimen, this scheme is known as Simultaneous Modulated Accelerated Radiation Therapy (SMART). These treatments require a strict quality control (QC) that should be done, ideally, in three dimensions. A potential tool for tridimensional dosimetry (3D) is the use of polymeric gel with Nuclear Magnetic Resonance Images (NMRI). In this paper, the 3D dosimetry with MAGIC-f gel was applied in 10 QC radiotherapy planning using the SMART technique. Initially, the dose distributions of assessed plans were remeasured in simulator geometry based on a CT scan. All plans used were previously approved in punctual and planar conventional CQ, which validate

In [231]:
# batch of size 64
batch = [description for _ in range(64)]
import time
s = time.time()
vectors = model.encode(batch)
e = time.time()
print(e - s, " segundos")

0.30879807472229004  segundos


In [232]:
vectors[0]

array([-1.08015187e-01, -1.34727880e-01,  9.88700032e-01,  2.80515075e-01,
       -2.15342253e-01, -1.13020754e+00,  3.06045651e-01, -6.51077747e-01,
        7.72035122e-01, -7.36724675e-01,  2.39772066e-01,  7.70318270e-01,
       -6.47946775e-01, -2.83951283e-01, -1.04673631e-01,  3.72441232e-01,
        4.39589858e-01, -2.13775799e-01, -6.21739745e-01,  6.43899068e-02,
        8.67942721e-02, -5.90848923e-01, -2.73701847e-01,  5.83076835e-01,
        2.92302907e-01,  2.78269619e-01,  6.45409107e-01,  1.20872736e+00,
        8.31682444e-01, -6.39874712e-02,  4.47392523e-01, -1.04317904e+00,
       -1.33988631e+00, -3.54544163e-01,  5.61213613e-01,  1.06995273e+00,
        4.03909177e-01, -5.88736653e-01, -6.07190609e-01, -8.25092345e-02,
        1.58240825e-01,  2.88934886e-01,  3.73817086e-01, -2.66998559e-01,
       -6.15275502e-01,  2.76715428e-01, -8.85831952e-01,  2.65091538e-01,
       -7.89880574e-01, -7.02626586e-01, -2.08835810e-01, -8.34421754e-01,
       -9.83947814e-02, -

In [236]:
# o primeiro texto é representado por
print(len(vectors[0]), "embeddings")

768 embeddings


#### Cálculo dos vetores

In [237]:
# encoding in batches can reduce overhead costs and speed up the process
vectors = []
batch_size = 64  # TODO: how can I speed up? Maybe reduce batch?
batch = []
for row in tqdm(data.itertuples()):
    description = row.title_en + " " + row.abstract_en + " " + row.keywords_en
    batch.append(description)
    if len(batch) >= batch_size:
        vectors.append(model.encode(batch))
        batch = []

if len(batch) > 0:
    vectors.append(model.encode(batch))
    batch = []

vectors = np.concatenate(vectors)

0it [00:00, ?it/s]

In [238]:
assert vectors.shape[0] == data.shape[0]

In [239]:
vectors.shape

(1329, 768)

In [240]:
np.save("../data/vectors.npy", vectors, allow_pickle=False)

### Testes - Para ter certeza de que os vetores fazem sentido

In [3]:
vectors = np.load("../data/vectors.npy")

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

> **Pergunta**: Como funciona cosine_similarity?

In [249]:
sample_query = "misconceptions about dengue"
# sample_query = "cancer in pregnancy"

In [250]:
print(sample_query)

misconceptions about dengue


In [251]:
query_vector = model.encode(sample_query)

In [252]:
[scores] = cosine_similarity([query_vector], vectors)

In [253]:
top_scores_ids = np.argsort(scores)

In [254]:
top_scores_ids = top_scores_ids[-5:][::-1]  # top 5 largest scores

In [255]:
for top_id in top_scores_ids:
    tmp = df.iloc[top_id]
    print("Index: ", tmp['index'])
    print("Title: ", tmp.title_en)
    print("Abstract: ", tmp.abstract_en)
    print("Keywords: ", tmp.keywords_en)
    print("#########################")

Index:  tde-04012017-101109
Title:  Knowledge, beliefs and practices regarding dengue
Abstract:  INTRODUCTION: Developing a better understanding of the knowledge and practices about dengue is essential to planning programs and public policies about the prevention of the disease. This research aimed to investigate the knowledge of users of primary health care services, living in Ribeirão Preto city, about dengue and its vector (transmission, symptoms, prevention and treatment), the source of the information, practices, and their associations with the knowledge and beliefs for the prophylaxis and treatment of disease. METHODS: This cross-sectional questionnaire survey of a sample of 605 people living in Ribeirão Preto was conducted following a major dengue outbreak in 2013. The respondents were invited to participate in the study at the time when they were waiting for medical attendance in the health units. RESULTS: The main source of information about dengue was from the television (87.